# Dogs vs Cat Redux

---

Notebook to finetune different architectures. Using full dataset.

In [3]:
from utils import *
from vgg16 import Vgg16
from keras.callbacks import ModelCheckpoint
from keras.initializers import he_normal
%matplotlib inline

Using cuDNN version 5005 on context None
Mapped name None to device cuda: GeForce GTX 1060 6GB (0000:04:00.0)
Using Theano backend.


Plan:

1. create decent size sample to work through ideas quickly
2. once a good approach is found automate it for full dataset
2. extra! create an ensemble with the sample data frist
3. __create a finetuned model of the best proto on full data__
3. submit to kaggle 3x times tonight

In [5]:
def frozen_vggbn():
    vggbn = Vgg16BN()
    vggbn = vggbn.model
    for layer in vggbn.layers:
        layer.trainable = False
    return vggbn

In [6]:
def half_frozen_vggbn(depth):
    '''Transfer learning from vggbn to a frozen model based on given depth.
    '''
    vggbn = Vgg16BN()
    vggbn = vggbn.model
    vggbn.pop()
    for layer in vggbn.layers[:depth]:
        #print('Freezing {}'.format(layer.name))
        layer.trainable = False
    for layer in vggbn.layers[depth:]:
        if 'dense' in layer.name:
            #print('Changing {} to he_normal initilizer'.format(layer.name))
            layer.kernel_initializer = he_normal()
    return vggbn

---
Save results get data from appropriate places

In [7]:
sample_train_path = '../../dogscats/lrg_sample/train/'
sample_val_path = '../../dogscats/lrg_sample/valid/'

train_path = '../../dogscats/train/'
val_path = '../../dogscats/valid/'

sample_results_path = '../../dogscats/lrg_sample/results/'
sample_model_path = '../../dogscats/lrg_sample/models/'

model_path = '../../dogscats/models/'

---

Set up data batches. Used both for prototyping but found that augmented is always results in better performance. 

In [8]:
aug_gen = image.ImageDataGenerator(
    channel_shift_range=10,
    zoom_range=0.05,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True)

In [9]:
batch_size=32 #32 - kept running out of mem on proto6

train_batches = get_batches(train_path,batch_size=batch_size)
aug_train_batches = aug_gen.flow_from_directory(directory=train_path,batch_size=batch_size,shuffle=True,target_size=(224,224))
val_batches = get_batches(val_path,batch_size=batch_size*2)

train_steps = train_batches.samples//train_batches.batch_size
aug_train_steps = aug_train_batches.samples//aug_train_batches.batch_size
val_steps = val_batches.samples//val_batches.batch_size

Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


---
Ensemble time.

In [10]:
def get_proto8():
    vggbn = half_frozen_vggbn(-14)
    for i in range(5):
        vggbn.pop()
    vggbn.add(BatchNormalization())
    vggbn.add(Dropout(.6))
    vggbn.add(Dense(2,activation='softmax',kernel_initializer='he_normal'))
    vggbn.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return vggbn

In [11]:
def get_proto4():
    vggbn = frozen_vggbn()
    vggbn.pop()
    vggbn.add(Dense(2,activation='softmax',kernel_initializer='he_normal'))
    vggbn.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return vggbn

In [12]:
def train_model(model_func,i,epochs=10):
    model = model_func()
    cb = [ModelCheckpoint(model_path+'lone_{}_{}'.format(model_name,i), monitor='val_loss', save_best_only=True, save_weights_only=False)]
    model.fit_generator(aug_train_batches, aug_train_steps, epochs=epochs, callbacks=cb,
                    validation_data=val_batches, validation_steps=val_steps)
    return model

---
Finetuning proto8 model from above with full data and modifying the learning rate. In addition, it seems to be overfitting quite highly now, so I'm going tp up the dropout from .5 to .6

I accidentally overwrote the ens_prod8_2 data with the single model. Thast why I'm loading it in as opposed to a lone_prod8_2

    LD score of .08891 (dropout .5)

In [13]:
model_name = 'prod8'

In [11]:
model_num = 2
model = train_model(get_proto8,model_num,epochs=10)

model.load_weights(model_path+'ens_{}_{}'.format(model_name,model_num))

test_batches = get_batches('../../dogscats/test/',batch_size=batch_size,shuffle=False)
test_steps = test_batches.n//test_batches.batch_size+1
y_pred = model.predict_generator(test_batches,steps=test_steps)

y_pred = pd.DataFrame(y_pred)
y_pred.to_csv(model_path+'lone_{}_{}_pred'.format(model_name,model_num))

Epoch 1/10
718/718 [==============================] - 351s - loss: 0.1279 - acc: 0.9543 - val_loss: 0.1830 - val_acc: 0.9466
Epoch 2/10
718/718 [==============================] - 339s - loss: 0.0845 - acc: 0.9694 - val_loss: 0.0599 - val_acc: 0.9783
Epoch 3/10
718/718 [==============================] - 339s - loss: 0.0598 - acc: 0.9785 - val_loss: 0.0485 - val_acc: 0.9850
Epoch 4/10
718/718 [==============================] - 348s - loss: 0.0505 - acc: 0.9832 - val_loss: 0.0349 - val_acc: 0.9861
Epoch 5/10
718/718 [==============================] - 339s - loss: 0.0383 - acc: 0.9856 - val_loss: 0.0365 - val_acc: 0.9902
Epoch 6/10
718/718 [==============================] - 340s - loss: 0.0348 - acc: 0.9883 - val_loss: 0.0388 - val_acc: 0.9902
Epoch 7/10
718/718 [==============================] - 337s - loss: 0.0428 - acc: 0.9841 - val_loss: 0.0869 - val_acc: 0.9814
Epoch 8/10
718/718 [==============================] - 352s - loss: 0.0311 - acc: 0.9887 - val_loss: 0.0308 - val_acc: 0.9907


In [14]:
cb = [ModelCheckpoint(model_path+'ens_{}_{}'.format(model_name,2), monitor='val_loss', save_best_only=True, save_weights_only=False)]
model.fit_generator(aug_train_batches, aug_train_steps, epochs=10, callbacks=cb,
                    validation_data=val_batches, validation_steps=val_steps)

Epoch 1/10
718/718 [==============================] - 362s - loss: 0.0267 - acc: 0.9907 - val_loss: 0.0377 - val_acc: 0.9886
Epoch 2/10
718/718 [==============================] - 348s - loss: 0.0348 - acc: 0.9879 - val_loss: 0.0462 - val_acc: 0.9871
Epoch 3/10
718/718 [==============================] - 348s - loss: 0.0267 - acc: 0.9906 - val_loss: 0.0360 - val_acc: 0.9897
Epoch 4/10
718/718 [==============================] - 365s - loss: 0.0162 - acc: 0.9952 - val_loss: 0.0398 - val_acc: 0.9881
Epoch 5/10
718/718 [==============================] - 348s - loss: 0.0164 - acc: 0.9942 - val_loss: 0.0512 - val_acc: 0.9881
Epoch 6/10
718/718 [==============================] - 349s - loss: 0.0125 - acc: 0.9963 - val_loss: 0.0661 - val_acc: 0.9855
Epoch 7/10
718/718 [==============================] - 353s - loss: 0.0142 - acc: 0.9950 - val_loss: 0.0478 - val_acc: 0.9871
Epoch 8/10
718/718 [==============================] - 355s - loss: 0.0133 - acc: 0.9956 - val_loss: 0.0415 - val_acc: 0.9881


---
Bumped the dropout to .6 and retuned the model another 5 epochs

    LD score of x

In [ ]:
model_num = 2
model_name = 'prod8'
model = train_model(get_proto8,model_num,epochs=5)

model.load_weights(model_path+'ens_{}_{}'.format(model_name,model_num))

test_batches = get_batches('../../dogscats/test/',batch_size=batch_size,shuffle=False)
test_steps = test_batches.n//test_batches.batch_size+1
y_pred = model.predict_generator(test_batches,steps=test_steps)

y_pred = pd.DataFrame(y_pred)
y_pred.to_csv(model_path+'lone_{}_{}_pred'.format(model_name,model_num))

Epoch 1/5
718/718 [==============================] - 353s - loss: 0.1216 - acc: 0.9569 - val_loss: 0.0623 - val_acc: 0.9798
Epoch 2/5
718/718 [==============================] - 347s - loss: 0.0695 - acc: 0.9738 - val_loss: 0.0333 - val_acc: 0.9886
Epoch 3/5
718/718 [==============================] - 335s - loss: 0.0573 - acc: 0.9803 - val_loss: 0.0807 - val_acc: 0.9819
Epoch 4/5
717/718 [============================>.] - ETA: 0s - loss: 0.0528 - acc: 0.9822

---
Went rogue at night and wanted to run a 50e of the proto8, surprisingly did poorly in the submission. 

    LD score of .09176

In [11]:
model_num = 1
model = train_model(get_proto8,model_num,epochs=50)

hist = model.load_weights(model_path+'ens_{}_{}'.format(model_name,model_num))

test_batches = get_batches('../../dogscats/test/',batch_size=batch_size,shuffle=False)
test_steps = test_batches.n//test_batches.batch_size+1
y_pred = model.predict_generator(test_batches,steps=test_steps)

y_pred = pd.DataFrame(y_pred)
y_pred.to_csv(model_path+'lone_{}_{}_pred'.format(model_name,model_num))

Epoch 1/50
718/718 [==============================] - 360s - loss: 0.1513 - acc: 0.9442 - val_loss: 0.0972 - val_acc: 0.9688
Epoch 2/50
718/718 [==============================] - 334s - loss: 0.1116 - acc: 0.9580 - val_loss: 0.1723 - val_acc: 0.9576
Epoch 3/50
718/718 [==============================] - 333s - loss: 0.0932 - acc: 0.9660 - val_loss: 0.3273 - val_acc: 0.9127
Epoch 4/50
718/718 [==============================] - 333s - loss: 0.0994 - acc: 0.9636 - val_loss: 0.1061 - val_acc: 0.9597
Epoch 5/50
718/718 [==============================] - 341s - loss: 0.0716 - acc: 0.9744 - val_loss: 0.0565 - val_acc: 0.9778
Epoch 6/50
718/718 [==============================] - 333s - loss: 0.0642 - acc: 0.9776 - val_loss: 0.0609 - val_acc: 0.9824
Epoch 7/50
718/718 [==============================] - 334s - loss: 0.0516 - acc: 0.9823 - val_loss: 0.1017 - val_acc: 0.9690
Epoch 8/50
718/718 [==============================] - 341s - loss: 0.0482 - acc: 0.9830 - val_loss: 0.0422 - val_acc: 0.9861


---
Create final pred

In [15]:
model_name = 'prod8'
model_num = 2

In [18]:
model = get_proto8()
model.load_weights(model_path+'lone_{}_{}'.format(model_name,model_num))

test_batches = get_batches('../../dogscats/test/',batch_size=batch_size,shuffle=False)
test_steps = test_batches.n//test_batches.batch_size+1
y_pred = model.predict_generator(test_batches,steps=test_steps)

y_pred = pd.DataFrame(y_pred)
y_pred.to_csv(model_path+'lone_{}_{}_pred'.format(model_name,model_num))

Found 12500 images belonging to 1 classes.


---
Load saved predictions of models and submit

In [19]:
ens_preds = pd.DataFrame()
for i in range(model_num):
    df = pd.read_csv(model_path+'lone_{}_{}_pred'.format(model_name,i+1),index_col=0)
    ens_preds = pd.concat([ens_preds,df],axis=1)

In [20]:
dog_preds = ens_preds.drop('0',axis=1)
avg_preds = dog_preds.mean(axis=1)
avg_preds = np.array(avg_preds)
avg_preds = avg_preds.clip(min=0.05, max=0.95)
#avg_preds = avg_preds.clip(lower=0.05,upper=0.95)

In [21]:
filenames = test_batches.filenames
full_files = [f.split('/')[1] for f in filenames]
ids = [int(f.split('.')[0]) for f in full_files]
formatted = np.stack([ids,avg_preds], axis=1)

In [22]:
sub_path = '../../dogscats/submissions/'
np.savetxt(sub_path+'lone_{}_{}_pred.csv'.format(model_name,model_num), formatted, fmt='%d,%.5f', header='id,label', comments='')

### Download and submit:

---
Use this command to download file from server. Must be done from the client *not* from this notebook

    scp 96.237.225.57:/home/xbno/anaconda3/courses/dogscats/submissions/submission1.csv ~/Desktop/

